# Navigation

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

The environment is already saved in the Workspace and can be accessed at the file path provided below.  Please run the next code cell without making any changes.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# please do not modify the line below
env = UnityEnvironment(file_name="/data/Banana_Linux_NoVis/Banana.x86_64")

# use_cuda = torch.cuda.is_available()
# device   = torch.device("cuda" if use_cuda else "cpu")
# print(device)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [ 1.          0.          0.          0.          0.84408134  0.          0.
  1.          0.          0.0748472   0.          1.          0.          0.
  0.25755     1.          0.          0.          0.          0.74177343
  0.          1.          0.          0.          0.25854847  0.          0.
  1.          0.          0.09355672  0.          1.          0.          0.
  0.31969345  0.          0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agent while it is training**, and you should set `train_mode=True` to restart the environment.

In [5]:
# env_info = env.reset(train_mode=True)[brain_name] # reset the environment
# state = env_info.vector_observations[0]            # get the current state
# score = 0                                          # initialize the score
# while True:
#     action = np.random.randint(action_size)        # select an action
#     env_info = env.step(action)[brain_name]        # send the action to the environment
#     next_state = env_info.vector_observations[0]   # get the next state
#     reward = env_info.rewards[0]                   # get the reward
#     done = env_info.local_done[0]                  # see if episode has finished
#     score += reward                                # update the score
#     state = next_state                             # roll over the state to next time step
#     if done:                                       # exit loop if episode finished
#         break
    
# print("Score: {}".format(score))

When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agent while it is training.  However, **_after training the agent_**, you can download the saved model weights to watch the agent on your own machine! 

In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import gym
import random
from collections import namedtuple,deque

N_STATES = state_size
N_ACTIONS = action_size

BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the network
MEMORY_CAPACITY = 50

EPSILON = 0.9               # greedy policy
TARGET_REPLACE_ITER = 100
seed = 999

In [8]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
#         self.fc1 = nn.Linear(N_STATES, 100)
#         self.fc1.weight.data.normal_(0, 0.1)   # initialization
#         self.out = nn.Linear(100, N_ACTIONS)
#         self.out.weight.data.normal_(0, 0.1)   # initialization
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, action_size)
        
        
    def forward(self, x):
#         x = self.fc1(x)
#         x = F.relu(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        actions_values = self.out(x)
        return actions_values

In [9]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [10]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()

        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()
        
        # Replay memory
        # self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
        
        
    def choose_action(self, x):
        x = Variable(torch.unsqueeze(torch.FloatTensor(x), 0))
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy
            actions_value = self.eval_net.forward(x)
            #zed = actions_value
            #print("actions_value : " , actions_value)
            action = torch.max(actions_value, 1)[1].data.numpy()[0]     # return the argmax Saurabh
        else:   # random
            action = np.random.randint(0, N_ACTIONS)
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition 
        self.memory_counter += 1

    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = Variable(torch.FloatTensor(b_memory[:, :N_STATES]))
        b_a = Variable(torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int)))
        b_r = Variable(torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2]))
        b_s_ = Variable(torch.FloatTensor(b_memory[:, -N_STATES:]))

        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        #print('q_eval',q_eval , q_eval.size() )  ---   ### Saurabh
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        #print('q_next', q_next , q_next.size() , b_r , b_r.size() , "gamma" , GAMMA ) ### Saurabh
        mmmmmmmmm = q_next.detach().max(1)[0].unsqueeze(1)  

        #print('q_next --- ', q_next.size() , b_r.size() ,  'mmmm' ,mmmm ) ### Saurabh
        
        q_target = b_r + (GAMMA * mmmmmmmmm *(1 - done) ) # q_next.max(1)[0]   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


In [ ]:
dqn = DQN()

train = True
env_info = env.reset(train_mode=train)[brain_name] # reset the environment and start training
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score globally once
n_episodes=10000

scores = []                        # list containing scores from each episode
scores_window = deque(maxlen=50)  # last 50/100 scores
    
for i_episode in range(1, n_episodes+1):
#
    env_info = env.reset(train_mode=True)[brain_name] # reset the environment
    state = env_info.vector_observations[0]  
    score = 0  #initalize the score for every episodes
    while train:
        action = dqn.choose_action(state)           #np.random.randint(action_size) # select an action
        env_info = env.step(action)[brain_name]     # put the action to the environment
        next_state = env_info.vector_observations[0]   # from env - get the next state
        
        if next_state.all == state.all :
            print("No Change")
        
        reward = env_info.rewards[0]                   # get the reward only 1st one 
        done = env_info.local_done[0]                  # see if episode has finished
        score += reward                                # update the score

        dqn.store_transition(state, action, score, next_state)

        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(score, 2))

        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break

    scores_window.append(score)       # save most recent score
    scores.append(score)              # save most recent score
    if i_episode % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window))) 
    if len(scores) >= 100 and i_episode % 100 == 0:
        scores_filename = "./Scores_2Agent_BothState" +str(i_episode) + ".csv"
        np.savetxt(scores_filename, scores_window,  delimiter=",")          
    if np.mean(scores_window)>=13.0 and train:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
        torch.save(torch.save(dqn, './SavedModel01_final'))
        break    
    
    
print("Score: {}".format(score))
torch.save(dqn, './SavedModel01')


Ep:  1 | Ep_r:  -5.0
Ep:  2 | Ep_r:  0.0
Ep:  3 | Ep_r:  1.0
Ep:  4 | Ep_r:  -1.0
Ep:  5 | Ep_r:  0.0
Ep:  6 | Ep_r:  0.0
Ep:  7 | Ep_r:  0.0
Ep:  8 | Ep_r:  -1.0
Ep:  9 | Ep_r:  0.0
Ep:  10 | Ep_r:  -1.0
Ep:  11 | Ep_r:  0.0
Ep:  12 | Ep_r:  1.0
Ep:  13 | Ep_r:  0.0
Ep:  14 | Ep_r:  0.0
Ep:  15 | Ep_r:  -2.0
Ep:  16 | Ep_r:  0.0
Ep:  17 | Ep_r:  0.0
Ep:  18 | Ep_r:  0.0
Ep:  19 | Ep_r:  0.0
Ep:  20 | Ep_r:  0.0
Ep:  21 | Ep_r:  -3.0
Ep:  22 | Ep_r:  1.0
Ep:  23 | Ep_r:  -1.0
Ep:  24 | Ep_r:  0.0
Ep:  25 | Ep_r:  -1.0
Ep:  26 | Ep_r:  -1.0
Ep:  27 | Ep_r:  0.0
Ep:  28 | Ep_r:  -3.0
Ep:  29 | Ep_r:  -1.0
Ep:  30 | Ep_r:  -1.0
Ep:  31 | Ep_r:  -2.0
Ep:  32 | Ep_r:  0.0
Ep:  33 | Ep_r:  1.0
Ep:  34 | Ep_r:  1.0
Ep:  35 | Ep_r:  -1.0
Ep:  36 | Ep_r:  -4.0
Ep:  37 | Ep_r:  0.0
Ep:  38 | Ep_r:  1.0
Ep:  39 | Ep_r:  0.0
Ep:  40 | Ep_r:  -1.0
Ep:  41 | Ep_r:  0.0
Ep:  42 | Ep_r:  0.0
Ep:  43 | Ep_r:  0.0
Ep:  44 | Ep_r:  0.0
Ep:  45 | Ep_r:  1.0
Ep:  46 | Ep_r:  -2.0
Ep:  47 | Ep_r:  1.0
Ep:  

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()
plt.savefig('Average_score_over500_episodes_p1_v0.png')


In [ ]:
env.close()
print('closed')